In [1]:
import time, sys, os
import gc

from multiprocessing import Pool
from astropy import units as u      # 用于单位转换的包
from astropy.coordinates import SkyCoord
import numpy as np
import pandas as pd



In [5]:
def choose_coord(a, b):
    skycood = SkyCoord(ra=a*u.degree,dec=b*u.degree,frame='icrs')
    g = skycood.galactic
    w = g.b.deg
    return w

In [6]:
chunk_size = 1000000

file = r'../dr8_v1.1_LRS_wd.csv'
file_all = r'../dr8_v1.1_LRS_catalogue.csv'
count = 1
rows = 0

In [8]:
data = pd.read_csv(file_all)
data.shape

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (41) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(10927505, 49)

In [9]:
data['b'] = data.apply(lambda x: choose_coord(x["ra_obs"], x["dec_obs"]), axis=1)

KeyboardInterrupt: 

In [9]:
data = data[data['b']>45]
data.describe()

,obsid,gp_id,lmjd,mjd,spid,fiberid,ra_obs,dec_obs,snru,snrg,...,gaia_g_mean_mag,offsets,offsets_v,ra,dec,teff,teff_err,logg,logg_err,b
count,4.158000e+03,4.158000e+03,4158.000000,4158.000000,4158.000000,4158.000000,4158.000000,4158.000000,4158.000000,4158.000000,...,4151.000000,4158.000000,4158.000000,4158.000000,4158.000000,4158.000000,4158.000000,4158.000000,4158.000000,4158.000000
mean,4.345419e+08,1.916114e+18,57466.049062,57465.049062,8.462001,124.081530,186.742116,28.240781,-13.057802,-1.231332,...,-62.020288,0.002165,0.007314,186.742114,28.240782,15177.938177,-1982.695527,-2492.610432,-2498.465367,62.229907
std,2.224723e+08,1.359444e+18,814.128151,814.128151,4.460758,71.489132,28.436685,16.020332,438.640257,411.191561,...,889.653999,0.046479,0.172130,28.436684,16.020332,17018.696660,4486.947057,4332.947424,4329.567777,10.766452
min,8.106134e+06,1.831024e+07,55892.000000,55891.000000,1.000000,1.000000,140.514010,-8.243798,-9999.000000,-9999.000000,...,-9999.000000,0.000000,0.000000,140.514010,-8.243798,-9999.000000,-9999.000000,-9999.000000,-9999.000000,45.010620
25%,2.844373e+08,8.049410e+17,57015.000000,57014.000000,5.000000,63.000000,161.238160,16.952820,1.060000,4.080000,...,16.666653,0.000000,0.000000,161.238160,16.952820,9280.430000,46.280000,6.604000,0.004250,53.470856
50%,4.371032e+08,1.394487e+18,57457.000000,57456.000000,8.000000,122.000000,185.891330,29.750925,2.290000,7.865000,...,17.860548,0.000000,0.000000,185.891330,29.750925,17203.845000,212.735000,7.691000,0.037000,60.899838
75%,5.841161e+08,3.702837e+18,57919.000000,57918.000000,12.000000,185.000000,210.106730,39.224936,5.790000,17.340000,...,18.758817,0.000000,0.000000,210.106730,39.224936,24000.025000,494.302500,8.099000,0.087000,70.182016
max,8.236161e+08,6.340880e+18,58993.000000,58992.000000,16.000000,250.000000,244.909740,60.650070,131.450000,287.900000,...,20.743979,1.000000,5.090000,244.909740,60.650070,83271.300000,9999.990000,9.962000,35.222000,89.255399


In [2]:
data.info()

NameError: name 'data' is not defined

In [55]:
a = pd.DataFrame(columns=data.columns)

In [56]:
pd.concat([a,data])

,obsid,uid,gp_id,designation,obsdate,lmjd,mjd,planid,spid,fiberid,...,offsets,offsets_v,ra,dec,wd_subclass,teff,teff_err,logg,logg_err,b
0,101076,G12763484624375,2676893926929699200,J220838.58-013731.7,2011-10-24,55859,55858,F5902,1,76,...,0,0.0,332.160763,-1.625490,DA,-9999.00,-9999.00,-9999.000,-9999.000,-43.425168
1,101151,G12762587004699,2677056689010163456,J220823.66-011534.2,2011-10-24,55859,55858,F5902,1,151,...,0,0.0,332.098588,-1.259508,DA,24000.00,672.02,8.263,0.076,-43.154923
2,112211,G13351718912434,2679476512239443328,J221640.39+012741.2,2011-10-24,55859,55858,F5902,12,211,...,0,0.0,334.168324,1.461445,DA,17122.64,221.39,7.933,0.037,-43.063161
3,116148,G13358737732360,2683213511743993472,J220522.86+021837.5,2011-10-24,55859,55858,F5902,16,148,...,0,0.0,331.345250,2.310432,DA,14070.30,215.59,8.063,0.028,-40.352703
4,204023,G8933577968159,2497952567036766848,J025747.26-002737.2,2011-10-24,55859,55858,F5907,4,23,...,0,0.0,44.446950,-0.460360,DA,-9999.00,-9999.00,-9999.000,-9999.000,-49.620308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15611,823814206,L13969314261795,823814206,J182122.96+183353.4,2020-05-27,58997,58996,GACII277N17M1,14,206,...,0,0.0,275.345680,18.564854,DA,26489.80,693.50,7.782,0.092,14.803124
15612,824005215,L14519339570388,824005215,J162937.34+052355.4,2020-05-27,58997,58996,HD163300N063640M02,5,215,...,0,0.0,247.405603,5.398735,DAZ,16620.23,76.96,-9999.000,-9999.000,33.838033
15613,824006121,G14518559500900,4435598234807569536,J164145.72+053537.1,2020-05-27,58997,58996,HD163300N063640M02,6,121,...,0,0.0,250.440521,5.593662,DA,7100.12,45.07,8.647,0.047,31.285448
15614,824006212,G14518361692417,4435580573901845760,J163942.86+052615.0,2020-05-27,58997,58996,HD163300N063640M02,6,212,...,0,0.0,249.928587,5.437514,DA,33696.96,312.21,-9999.000,-9999.000,31.658385


In [9]:
data[['obsid','ra_obs','dec_obs']]

NameError: name 'data' is not defined

In [2]:


a = pd.read_csv('../dr8_gb_greater_45.csv')

In [11]:
a[['obsid','ra_obs','dec_obs']]

,obsid,ra_obs,dec_obs
0,3706051,141.906380,27.271168
1,3706056,141.872385,27.354952
2,3706057,141.985427,27.195557
3,3706059,142.127981,27.156590
4,3706060,141.913714,27.183941
...,...,...,...
2586082,823113215,189.614061,33.370878
2586083,823113217,190.085984,33.654721
2586084,823113219,189.870454,33.494932
2586085,823113222,189.905752,33.559725


In [4]:
b=a[['obsid','ra_obs','dec_obs']]

In [7]:
b[b['obsid']==3706051]

,obsid,ra_obs,dec_obs
0,3706051,141.90638,27.271168


In [13]:
b.to_csv('../coord_greater_45.csv')

In [3]:
sum(a[a['class']=='GALAXY']['z']<0.3)  # all 122114

121350

In [11]:
a[(a['class']=='QSO') & (a['z']!=-9999.0) &(a['z']<0.3)]['z'].describe()

count    1876.000000
mean        0.185526
std         0.075202
min        -0.004371
25%         0.126650
50%         0.190185
75%         0.252276
max         0.299978
Name: z, dtype: float64

In [12]:
a[(a['class']=='QSO') & (a['z']!=-9999.0) &(a['z']<0.3) &(a['z']>0)]['z'].describe()

count    1873.000000
mean        0.185830
std         0.074878
min         0.000209
25%         0.127318
50%         0.190607
75%         0.252356
max         0.299978
Name: z, dtype: float64

In [37]:
q = a[(a['class']=='QSO') & (a['z']!=-9999.0)]


In [38]:
g = a[(a['class']=='GALAXY') & (a['z']!=-9999.0)]
g = g[g['z']<0.12]

In [41]:
g['z'].describe()

count    86397.000000
mean         0.071075
std          0.027556
min         -0.004021
25%          0.051145
50%          0.073082
75%          0.091686
max          0.119999
Name: z, dtype: float64

## 去红移

In [42]:
from astropy.io import fits
import numpy as np

In [44]:
fits_path = '/home/shichenhui/code/data/data_process/constract_dataset/spec-55859-F5902_sp02-140.fits.gz'
fits_file = fits.open(fits_path)
hdu = fits_file[0]
data = hdu.data[0]

In [65]:
coeff0 = hdu.header['COEFF0']
z=hdu.header['z']
coeff0

3.5682

In [47]:
10**3.5682

3699.9853117285957

In [48]:
len(data)

3890

In [60]:
# numpy.linspace(start, stop, num=50, endpoint=True, retstep=False, dtype=None, axis=0)
wave = np.linspace(start=coeff0,stop=coeff0+0.0001*len(data),num=len(data),endpoint=False)

In [83]:

10**wave,10**wave/(1+0.3)

(array([3699.98531173, 3700.83736292, 3701.68961033, ..., 9055.24072268,
        9057.32600898, 9059.41177549]),
 array([2846.14254748, 2846.79797148, 2847.45354641, ..., 6965.56978668,
        6967.17385306, 6968.77828884]))

In [84]:
wr = 10**wave/(1+0.3)

In [92]:
len(wr[(wr>3800) & (wr<6960)])

2628

In [93]:
w=10**wave

In [94]:
len(w[(w>3800) & (w<6960)])

2629

In [96]:
np.log10(6960*1.3)-np.log10(3800*1.3)

0.2628256429937519